# Jupyter Notebook approach for SCORE interaction


### Necessary packages 

In [1]:
from checkscore.methods import readonly, external, payable
from checkscore.methods import Checkscore
from iconsdk.wallet.wallet import KeyWallet

password = input()
deployer_wallet = KeyWallet.load("keystore_ibriz.json",password)
deployer_wallet.get_address()

SCORE_ADDRESS = ""
Checkscore(SCORE_ADDRESS).fill_methods()

ModuleNotFoundError: No module named 'checkscore'

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [14]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
import requests
import json
import pickle
import csv
import os
from pprint import pprint
from getpass import getpass

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

mainnet= {
    "iconservice":"https://ctz.solidwallet.io",
    "nid" :1
    
}

yeouido={
     "iconservice":"https://bicon.net.solidwallet.io",
     "nid" : 3
    
}
euljiro={
     "iconservice":"https://test-ctz.solidwallet.io",
     "nid" : 2
    
}
pagoda={
    "iconservice":"https://zicon.net.solidwallet.io",
     "nid" :80
    
    
}
custom={
    "iconservice":"http://18.237.205.52:9000/",
     "nid":3
    
    
}


### Required service

In [16]:
env = yeouido #mainnet,yeouido,euljiro,pagoda,custom
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]



### A list for SCORE addresses 

In [21]:
# Yeouido Testnet
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"

USDb = "cx19be1be7b4e7750863edf966faac3f42dec21025"
oUSDb = "cx4258464e395b3ef3b73a146947eb41393e8c5873"
LendingPool= "cx35b12ca4e9c672518db3c06d92ed60d50e7d5554"
LendingPoolCore = "cx327a386a9f8076045679fc132a8baa9e270ca43b"
LendingPoolDataProvider = "cxa198eaf0a589adf0ab31b523d1f8912f47f6f293"
PriceOracle = "cx3f6477bf290eb91257b806ece614ab18b1c7be41"
AddressProvider = "cxed440f1da2cf3087ac5ef12217a13f6321de0375"
FeeProvider = "cxede316aa65b7df13633599e67490daf1fd32b8a5"

# Mainnet}
# SCORE_ADDRESS = "cx346a2c30ea2712557deb4d7392688750350d1dea"

In [18]:

# Custom
USDb = "cx072d8f95877a53df350e3dc3d8dba2f379037d42"
oUSDb = "cx48dd26868c25cfa744aab9f2fccd76ed40c41454"
LendingPool= "cxbf1a257dcb378b0fd41066fa87abef3f25992832"
LendingPoolCore = "cx162fe5b5d3276e9493de1fd440511f0844ca9a7c"
LendingPoolDataProvider = "cx84490f03cc04b799fe668545340eaeac7cae628c"
PriceOracle = "cx293582520a75648d91a3c454ad9657aa4cbf1f7d"
AddressProvider = "cxad29953f5b8957704e9092632bccc3b1aac62eea"
FeeProvider = "cxf293e454a0c8a28142da48a136458a177d047769"


### Importing necessary wallets

If you want to use your pre-exisiting wallets, execute the cell below:-

In [20]:
password = getpass()

deployer_wallet = KeyWallet.load("keystore_2",password)
deployer_wallet.get_address()


········


'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'

### OR creating new wallets 

If you want to use new wallets, execute the cell below:-

In [73]:

caller_wallet = KeyWallet.create()
print(caller_wallet.get_address())
caller_wallet.get_private_key()

hx1bac45f9b4ed34edf99c804b84de69f8d6248e48


'66a942ac4b994a5c3eef1bef331c4bbd488a86ec4db5d3f20c28d9c8706c1e29'

### Loading ICX
Go to [ibriz faucet](https://icon-faucet.ibriz.ai/) to load ICX into your wallets.

### Deploying contract

In [29]:
# params = {'_initialSupply':500000000, '_decimals': 18}
deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cxad29953f5b8957704e9092632bccc3b1aac62eea")\
    .nid(3)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('feeProvider'))\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0xaf225f085cf3c8140e0f99d38cba21ed94e85ad73cdcb70d6267ec8eed20e754',
 'blockHeight': 229676,
 'blockHash': '0x5817cc5e2183181f5263fa19852a1340d4fde0370ba1bcad0d64ed720413fe0f',
 'txIndex': 1,
 'to': 'cx0000000000000000000000000000000000000000',
 'scoreAddress': 'cxf293e454a0c8a28142da48a136458a177d047769',
 'stepUsed': 1040693600,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1040693600,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

### Updating contract

In [68]:
deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(LendingPoolDataProvider)\
    .nid(NID)\
    .nonce(1000)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('lendingPoolDataProvider'))\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)


get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x0bf06b340d4036e37935396f52bec3f6ea43c21565ce247dc903438a7155cf93',
 'blockHeight': 8960043,
 'blockHash': '0x6c3151a595fec4323cc0a35c88af860d8f9c95d70aaa1a9521f825ff28f28f32',
 'txIndex': 1,
 'to': 'cxa198eaf0a589adf0ab31b523d1f8912f47f6f293',
 'scoreAddress': 'cxa198eaf0a589adf0ab31b523d1f8912f47f6f293',
 'stepUsed': 1698757600,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1698757600,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

## Adding new reserve in LendingPoolCore



In [ ]:
params ={"_reserve": {"reserveAddress":"cx072d8f95877a53df350e3dc3d8dba2f379037d42","oTokenAddress":"cx48dd26868c25cfa744aab9f2fccd76ed40c41454","totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"60","liquidationThreshold":"65","liquidationBonus":"10","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx162fe5b5d3276e9493de1fd440511f0844ca9a7c") \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

## Setting reserve constants in LendingPoolCore

In [ ]:
params ={"_constants": [{"reserve":"cx072d8f95877a53df350e3dc3d8dba2f379037d42","optimalUtilizationRate":f"6{'0'*17}","baseBorrowRate":"0","slopeRate1":f"7{'0'*16}","slopeRate2":f"3{'0'*18}"} ]}
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx162fe5b5d3276e9493de1fd440511f0844ca9a7c") \
    .nid(NID) \
    .nonce(100) \
    .method("setReserveConstants")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

## Setting interfaces in LendingPool

In [42]:
params ={"_address": LendingPool }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(oUSDb) \
    .nid(NID) \
    .nonce(100) \
    .method("setLendingPoolAddress")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx48dd26868c25cfa744aab9f2fccd76ed40c41454', 'stepLimit': '0x3c460', 'timestamp': '0x5b347ebaa9004', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'setLendingPoolAddress', 'params': {'_address': 'cxbf1a257dcb378b0fd41066fa87abef3f25992832'}}, 'signature': '6qs4PAjl7bIzL2CxbtqhyZCkJdEfsF9wa6UZ1RF3kyEsGHybc1cLI4sG0nTkMvHhx6hlwkUhrw0scAZCuKTp+wA='}


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0xefefd9c2ee60b913298a043bf5e9a7654baa50861f8adb19c71e11d1849c5e65',
 'blockHeight': 230186,
 'blockHash': '0xb90b4dc6ee9931d2b47eebfa93e6504f1d5d186f19ff2932a6b6d0835f658b01',
 'txIndex': 1,
 'to': 'cx48dd26868c25cfa744aab9f2fccd76ed40c41454',
 'stepUsed': 146880,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 146880,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [12]:
params ={"_address": "cx072d8f95877a53df350e3dc3d8dba2f379037d42" }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cxbf1a257dcb378b0fd41066fa87abef3f25992832") \
    .nid(NID) \
    .nonce(100) \
    .method("setUSDbAddress")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cxbf1a257dcb378b0fd41066fa87abef3f25992832', 'stepLimit': '0x3d298', 'timestamp': '0x5b309044d16af', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'setUSDbAddress', 'params': {'_address': 'cx072d8f95877a53df350e3dc3d8dba2f379037d42'}}, 'signature': 'QudO9QJkDPtvc5UFbQIgBvjDMc/Vj7RJ5zlKcljU6PU9qKdNNaXZIVLeuAWsCCoNGLaFg88mm+pt3L7QhVeDFgE='}


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x6d25b73a67de8cc1c15422d1bd91b24d987fd7049fe1c2ff095231082048ef2b',
 'blockHeight': 95270,
 'blockHash': '0xa6c15d3ae47aeac3a6d0159aebb685bb1cadad28aee31bf411631542e1f621db',
 'txIndex': 1,
 'to': 'cxbf1a257dcb378b0fd41066fa87abef3f25992832',
 'stepUsed': 150520,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 150520,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

## Setting interfaces in LendingPoolDataProvider

In [ ]:
params ={"_address": "cx162fe5b5d3276e9493de1fd440511f0844ca9a7c" }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx84490f03cc04b799fe668545340eaeac7cae628c") \
    .nid(NID) \
    .nonce(100) \
    .method("setLendingPoolCoreAddress")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

In [66]:
params ={"_reserveAddress": USDb, "_liquidationThreshold": 65 * 10**16 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(LendingPoolCore) \
    .nid(NID) \
    .nonce(100) \
    .method("updateLiquidationThreshold")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)



# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x14a010da9a2f5329cbed4dfc25bd15aae012686714ca449593300db492a66dd2',
 'blockHeight': 8959753,
 'blockHash': '0xac1af9986a9417af4f6acde9ea2eebccaa1f327bd151e8b428d13832a14d059f',
 'txIndex': 1,
 'to': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
 'stepUsed': 157040,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 157040,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

## Setting Price in Oracle

In [ ]:
params ={"_address": "cx162fe5b5d3276e9493de1fd440511f0844ca9a7c" }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx84490f03cc04b799fe668545340eaeac7cae628c") \
    .nid(NID) \
    .nonce(100) \
    .method("setLendingPoolCoreAddress")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

In [ ]:
params ={"_reserveAddress": "cx072d8f95877a53df350e3dc3d8dba2f379037d42","_liquidityRate":10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx162fe5b5d3276e9493de1fd440511f0844ca9a7c") \
    .nid(NID) \
    .nonce(100) \
    .method("updateLiquidityRate")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

## Setting Price in Oracle

In [34]:
params ={"_base": "USDb", "_quote":"USD","_rate": 5 * 10**17 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx293582520a75648d91a3c454ad9657aa4cbf1f7d") \
    .nid(NID) \
    .nonce(100) \
    .method("set_reference_data")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx293582520a75648d91a3c454ad9657aa4cbf1f7d', 'stepLimit': '0x3bf88', 'timestamp': '0x5b359d9761695', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'set_reference_data', 'params': {'_base': 'USDb', '_quote': 'USD', '_rate': '0x6f05b59d3b20000'}}, 'signature': 'xoc94hdu+Sc6ZdL31oIjltkGCRJFGbg9ogT6WSoB/DsDQuS4iQtvGzGVN9t5sddKk6VtKBV38v4KGaTRSXVb6AA='}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x88f684bc4419ee00c4d3bfbec318f38a822764693bbadba8c5fc38a3b74bf093',
 'blockHeight': 268640,
 'blockHash': '0x4c2025fa7a09f4b9687b0550bdefd2b7b523c63df44989e0332e19512232ea73',
 'txIndex': 1,
 'to': 'cx293582520a75648d91a3c454ad9657aa4cbf1f7d',
 'stepUsed': 145640,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 145640,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

## Withdraw USDb

In [ ]:
params = {"_amount": 5000000000000000000}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(oUSDb)\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("redeem")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

## Deposit USDb

In [75]:

data = "{\"method\": \"deposit\", \"params\": {\"amount\": 100000000000000000000}}".encode("utf-8")
params = {"_to": LendingPool, "_value": 100000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(caller_wallet.get_address())\
    .to(USDb)\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, caller_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x4ac01639baf90396b384f70fe7d0a4a6ea053773e1ae6f6a3c592c2643cb5aec',
 'blockHeight': 8961963,
 'blockHash': '0x5aed46354f63990a4fe1cff75ffd43b1b688349a3b96867ef56b58865877d113',
 'txIndex': 1,
 'to': 'cx19be1be7b4e7750863edf966faac3f42dec21025',
 'stepUsed': 670600,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 670600,
 'eventLogs': [{'scoreAddress': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cx19be1be7b4e7750863edf966faac3f42dec21025',
    '0xa103b96d65',
    '0x1104f8dff271d'],
   'data': ['0xde0b6b3af6b0958', '0xde0b6c008d2fc03']},
  {'scoreAddress': 'cx4258464e395b3ef3b73a146947eb41393e8c5873',
   'indexed': ['Mint(Address,int)',
    'hx1bac45f9b4ed34edf99c804b84de69f8d6248e48'],
   'data': ['0x0']},
  {'scoreAddress': 'cx4258464e395b3ef3b73a146947eb41393e8c5873',
   'indexed': ['Mint(Address,int)',
    'hx1bac45f9b4ed34edf99c804b84de69f8d6248e48'],
   'data': ['0x56bc75e2d63100000']},
  {'scoreAddres

## Borrow USDb

In [54]:
params = {"_amount": 100*10**18 ,"_reserve": USDb}
transaction = CallTransactionBuilder()\
    .from_(caller_wallet.get_address())\
    .to(LendingPool)\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("borrow")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, caller_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'txHash': '0x406b195d87747b76b10c6c630bbb3cd37ab4796a82d488696cf215d8182ce3b1',
 'blockHeight': 8959268,
 'blockHash': '0x7527f1b652570a5247efb5e7b848c9ebe698ae2454981437719248674c4b101c',
 'txIndex': 1,
 'to': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
 'stepUsed': 1055660,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1055660,
 'eventLogs': [{'scoreAddress': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cx19be1be7b4e7750863edf966faac3f42dec21025',
    '0x226ae6ccd89',
    '0x1f7989f47126d'],
   'data': ['0xde0b6b3a7640000', '0xde0b6b3a7640000']},
  {'scoreAddress': 'cx19be1be7b4e7750863edf966faac3f42dec21025',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'cx327a386a9f8076045679fc132a8baa9e270ca43b',
    'hx80d817397f58993db61b2fddcf1a9207669419ed',
    '0x56bc75e2d63100000'],
   'data': ['0x4e6f6e65']}],
 'logsBloom': b'\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

## Repay USDb

In [76]:

data = "{\"method\": \"repay\", \"params\": {\"amount\": 5000000000000000000}}".encode("utf-8")
params = {"_to": LendingPool, "_value": 5000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(caller_wallet.get_address())\
    .to(USDb)\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, caller_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xb0252fac4d540253fb0f32343d2265350514b5d6909b0df46067b503b74cd255',
 'blockHeight': 8962605,
 'blockHash': '0x158a7d480f40cc38cdb8481692c390ad084971d79c499d8e5503e8c850d4a25b',
 'txIndex': 1,
 'to': 'cx19be1be7b4e7750863edf966faac3f42dec21025',
 'stepUsed': 851040,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 851040,
 'eventLogs': [{'scoreAddress': 'cx327a386a9f8076045679fc132a8baa9e270ca43b',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cx19be1be7b4e7750863edf966faac3f42dec21025',
    '0xc0b1553600',
    '0x129e569ba971d'],
   'data': ['0xde0b6b3b1256283', '0xde0b6c2e9ddceda']},
  {'scoreAddress': 'cx19be1be7b4e7750863edf966faac3f42dec21025',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
    'cx327a386a9f8076045679fc132a8baa9e270ca43b',
    '0x4563918244f40000'],
   'data': ['0x4e6f6e65']},
  {'scoreAddress': 'cx35b12ca4e9c672518db3c06d92ed60d50e7d5554',
   'indexed': ['Repay(Address,Address,

### ICX Transfer

In [8]:
transaction = TransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("hx177b10efe3018961a405cc7c20ee811c552831a5")\
    .nid(3) \
    .nonce(100) \
    .value(10000*10**18)\
    .build()

estimate_step = icon_service.estimate_step(transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'hx177b10efe3018961a405cc7c20ee811c552831a5', 'stepLimit': '0x30d40', 'timestamp': '0x5b352f9520905', 'nid': '0x3', 'value': '0x10f0cf064dd59200000', 'nonce': '0x64', 'signature': 'k2uHKccWbioV8N5TkAXuffsOprQ90Jb5nMD9M2u3PiBE1WO3h2c0zzDsmNR3/8nq2Ypz20vvh2bVFm2L1kQhFAE='}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x8f67dc35d0b967bdac94eb8431b7956c0f21d72de631f6e5538cd01612c0379c',
 'blockHeight': 253893,
 'blockHash': '0xde3f86e1c017936f5a60e9ddf7381590423c9c69a440e1420fe42036acea9a9b',
 'txIndex': 1,
 'to': 'hx177b10efe3018961a405cc7c20ee811c552831a5',
 'stepUsed': 100000,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 100000,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [10]:
# hx91bf040426f226b3bfcd2f0b5967bbb0320525ce
balance = icon_service.get_balance("hx91bf040426f226b3bfcd2f0b5967bbb0320525ce")
print(balance/10**18)

78735532.9413376


In [ ]:

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to()\
    .method("getAllAddress")\
    .build()
response = icon_service.call(_call)
print(response)

In [65]:
params = {'_user': "hx80d817397f58993db61b2fddcf1a9207669419ed" }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(LendingPoolDataProvider)\
    .method("getUserAccountData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if values=="0x1" or values =="0x0" or "Address" in key:
        continue
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

{'currentLiquidationThreshold': 3.25e-13,
 'currentLtv': 60.0,
 'healthFactor': 646766.1574126608,
 'healthFactorBelowThreshold': '0x0',
 'totalBorrowBalanceUSD': 25.125000457107184,
 'totalCollateralBalanceUSD': 5000.0000001952485,
 'totalFeesUSD': '0x0',
 'totalLiquidityBalanceUSD': 5000.0000001952485}


In [61]:
params = {'_user': 'hx80d817397f58993db61b2fddcf1a9207669419ed' ,"_reserve":USDb}

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(LendingPoolDataProvider)\
    .method("getUserReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
print(response)

{'currentOTokenBalance': '0x21e19e0ca0a27b7ec90', 'currentOTokenBalanceUSD': '0x10f0cf0650513dbf648', 'currentBorrowBalance': '0x2b95bdd7da1135da4', 'currentBorrowBalanceUSD': '0x15cadeebed089aed2', 'principalBorrowBalance': '0x2b95bdd7bfa52c388', 'principalBorrowBalanceUSD': '0x15cadeebdfd2961c4', 'borrowRate': '0xfd05641cbb79', 'liquidityRate': '0x8b02ca38c2', 'originationFee': '0x0', 'userBorrowCumulativeIndex': '0xde0b6b57f614c32', 'lastUpdateTimestamp': '0x5b3ae966761d5', 'useAsCollateral': '0x1'}


In [63]:
int("0x2b95bdd7da1135da4",0)

50250000798959492516

In [67]:
params = {"_reserve": USDb}
# params={"_base":"USDb","_quote":"USD"}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(LendingPoolCore)\
    .method("getReserveConfiguration")\
    .params(params)\
    .build()
response = icon_service.call(_call)
print(response)


{'decimals': '0x12', 'baseLTVasCollateral': '0x340aad21b3b700000', 'liquidationThreshold': '0x905438e60010000', 'usageAsCollateralEnabled': '0x1', 'isActive': '0x1', 'borrowingEnabled': '0x1', 'liquidationBonus': '0xa'}


In [14]:
params ={"_reserveAddress":"cx072d8f95877a53df350e3dc3d8dba2f379037d42","_baseLTVasCollateral": 60*10**18}
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(LENDING_POOL_CORE) \
    .nid(NID) \
    .nonce(100) \
    .method("updateBaseLTVasCollateral")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx162fe5b5d3276e9493de1fd440511f0844ca9a7c', 'stepLimit': '0x3ec60', 'timestamp': '0x5b3599c7c61af', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'updateBaseLTVasCollateral', 'params': {'_reserveAddress': 'cx072d8f95877a53df350e3dc3d8dba2f379037d42', '_baseLTVasCollateral': '0x340aad21b3b700000'}}, 'signature': 'xoUfIP+7UtZMAIghMqcASInpDA+N2NMMJXGNIaPD8kktym68Bo2tQWKTrwQB78r6OXwXusEjnGbs/TnYPtgPYwA='}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x0c863fc561754b4681d7a368753631b20ed25c18555f6b347da4748ab3dd7978',
 'blockHeight': 268129,
 'blockHash': '0x3f57da6f1ed0df92ef0219cf075dd16cca215dd598494b2a8d1b2495c5be4066',
 'txIndex': 1,
 'to': 'cx162fe5b5d3276e9493de1fd440511f0844ca9a7c',
 'stepUsed': 157120,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 157120,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [74]:
params = {"_to": "hx1bac45f9b4ed34edf99c804b84de69f8d6248e48", "_value": 50000000000000000000000}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(USDb)\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xa3b6f17dd433f7690549965c5e904e02be2d82ca820a540f3bba677e72737686',
 'blockHeight': 8961324,
 'blockHash': '0x2e9911b1220e0359b8626bc957bace4d54afb80d5ed75658afe5a615a5d3c6d5',
 'txIndex': 1,
 'to': 'cx19be1be7b4e7750863edf966faac3f42dec21025',
 'stepUsed': 161460,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 161460,
 'eventLogs': [{'scoreAddress': 'cx19be1be7b4e7750863edf966faac3f42dec21025',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce',
    'hx1bac45f9b4ed34edf99c804b84de69f8d6248e48',
    '0xa968163f0a57b400000'],
   'data': ['0x4e6f6e65']}],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x